In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta


In [2]:
df = pd.read_csv('../data/processed/cleaned_transactions.csv', parse_dates=['InvoiceDate'])

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,TotalPrice,Year,Month,DayOfWeek,Hour
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,2009,12,1,7
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009,12,1,7
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009,12,1,7
3,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,2009,12,1,7
4,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085,United Kingdom,39.6,2009,12,1,7


In [3]:
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)
snapshot_date


Timestamp('2010-12-10 20:01:00')

In [4]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,   # Recency
    'InvoiceNo': 'nunique',                                     # Frequency
    'TotalPrice': 'sum'                                         # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
rfm.head()


,CustomerID,Recency,Frequency,Monetary
0,12346,165,11,372.86
1,12347,3,2,1224.17
2,12348,74,1,222.16
3,12349,43,2,1635.89
4,12351,11,1,288.18


In [5]:
additional_features = df.groupby('CustomerID').agg({
    'Quantity': ['sum', 'mean'],
    'TotalPrice': ['mean'],
    'InvoiceDate': ['min', 'max']
})

additional_features.columns = [
    'total_quantity',
    'avg_quantity',
    'avg_order_value',
    'first_purchase_date',
    'last_purchase_date'
]

additional_features = additional_features.reset_index()
additional_features.head()


,CustomerID,total_quantity,avg_quantity,avg_order_value,first_purchase_date,last_purchase_date
0,12346,70,2.121212,11.298788,2009-12-14 08:34:00,2010-06-28 13:53:00
1,12347,759,11.161765,18.002500,2010-10-31 14:20:00,2010-12-07 14:57:00
2,12348,373,18.650000,11.108000,2010-09-27 14:59:00,2010-09-27 14:59:00
3,12349,898,11.225000,20.448625,2010-04-29 13:20:00,2010-10-28 08:23:00
4,12351,260,13.000000,14.409000,2010-11-29 15:23:00,2010-11-29 15:23:00


In [6]:
customer_features = pd.merge(rfm, additional_features, on='CustomerID')
customer_features.head()


,CustomerID,Recency,Frequency,Monetary,total_quantity,avg_quantity,avg_order_value,first_purchase_date,last_purchase_date
0,12346,165,11,372.86,70,2.121212,11.298788,2009-12-14 08:34:00,2010-06-28 13:53:00
1,12347,3,2,1224.17,759,11.161765,18.002500,2010-10-31 14:20:00,2010-12-07 14:57:00
2,12348,74,1,222.16,373,18.650000,11.108000,2010-09-27 14:59:00,2010-09-27 14:59:00
3,12349,43,2,1635.89,898,11.225000,20.448625,2010-04-29 13:20:00,2010-10-28 08:23:00
4,12351,11,1,288.18,260,13.000000,14.409000,2010-11-29 15:23:00,2010-11-29 15:23:00


In [7]:
churn_threshold = 90  # days

customer_features['Churn'] = np.where(
    customer_features['Recency'] > churn_threshold,
    1,
    0
)

customer_features['Churn'].value_counts(normalize=True)


Churn
0    0.668116
1    0.331884
Name: proportion, dtype: float64

In [8]:
customer_features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4140 entries, 0 to 4139
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CustomerID           4140 non-null   int64         
 1   Recency              4140 non-null   int64         
 2   Frequency            4140 non-null   int64         
 3   Monetary             4140 non-null   float64       
 4   total_quantity       4140 non-null   int64         
 5   avg_quantity         4140 non-null   float64       
 6   avg_order_value      4140 non-null   float64       
 7   first_purchase_date  4140 non-null   datetime64[ns]
 8   last_purchase_date   4140 non-null   datetime64[ns]
 9   Churn                4140 non-null   int32         
dtypes: datetime64[ns](2), float64(3), int32(1), int64(4)
memory usage: 307.4 KB


In [9]:
customer_features.head()


,CustomerID,Recency,Frequency,Monetary,total_quantity,avg_quantity,avg_order_value,first_purchase_date,last_purchase_date,Churn
0,12346,165,11,372.86,70,2.121212,11.298788,2009-12-14 08:34:00,2010-06-28 13:53:00,1
1,12347,3,2,1224.17,759,11.161765,18.002500,2010-10-31 14:20:00,2010-12-07 14:57:00,0
2,12348,74,1,222.16,373,18.650000,11.108000,2010-09-27 14:59:00,2010-09-27 14:59:00,0
3,12349,43,2,1635.89,898,11.225000,20.448625,2010-04-29 13:20:00,2010-10-28 08:23:00,0
4,12351,11,1,288.18,260,13.000000,14.409000,2010-11-29 15:23:00,2010-11-29 15:23:00,0


In [10]:
customer_features.to_csv('../data/processed/customer_features.csv', index=False)
